# Data Capture

In [19]:
with open("/home/z3r0/Code/Python/Algorithmic_Complexity/TP/SF_street_intersections.csv") as f:
    data = f.readlines()[1:]
    intercepts = dict()
    streets = dict()
    nodes = dict()
    nodeToIntercept = dict()
    c = 0
    for line in data:
        street, point = line.strip().split(sep=',')[1:3]
        lat, lon = point[7:-2].split()
        intercept = (float(lat), float(lon))
        if street in intercepts.keys():
            intercepts[street].append(intercept)
        else:
            intercepts[street] = [intercept]
        if intercept in streets.keys():
            streets[intercept].append(street)
        else:
            streets[intercept] = [street]
            nodes[intercept] = c
            nodeToIntercept[c] = intercept
            c += 1

In [22]:
intercepts['UTAH']

[(-122.40545417189192, 37.7543372313743),
 (-122.40679858931149, 37.76841488233248),
 (-122.40606109793055, 37.7607529052339),
 (-122.4061819835959, 37.7619964030768),
 (-122.40655282446389, 37.7658422711544),
 (-122.40515881630326, 37.7512810744602),
 (-122.40593403378597, 37.7594458071505),
 (-122.40688239974939, 37.769298557428),
 (-122.40521698628322, 37.7562225803215),
 (-122.40642915445186, 37.7645476536961),
 (-122.40520770296072, 37.7517869480963),
 (-122.40630557076001, 37.7632538870406),
 (-122.40525954218414, 37.75692043860479),
 (-122.40533110370748, 37.753063831974),
 (-122.40667700476389, 37.7671421818598)]

In [23]:
streets[(-122.40545417189192, 37.7543372313743)]

['UTAH', '23RD']

# Graph creation

In [20]:
def createGraph(streets, intercepts, nodes):
    G = [[] for i in range(len(streets.keys()))]
    def calculateDistance(p1, p2):
        return sum((map(lambda c1, c2: abs(c1 - c2), intercept, point)))
    for intercept in streets.keys():
        for street in streets[intercept]:
            distance = []
            for point in intercepts[street]:
                if point != intercept:
                    # print(calculateDistance(intercept, point))
                    distance.append([nodes[point], calculateDistance(intercept, point)])
            # print(distance)
            distance.sort(key=lambda ls: ls[1])
            if len(distance) > 1:
                neighbours = distance[0:2]
                if len(neighbours) > 1 and calculateDistance(neighbours[0][1], neighbours[1][1]) > neighbours[0][1]:
                    neighbours.pop()
                G[nodes[intercept]].extend([neighbours[x][0] for x in range(len(neighbours))])
            elif len(distance) > 0:
                G[nodes[intercept]].append(distance[0][0])
    
    return G

In [8]:
G_al = createGraph(streets, intercepts, nodes)
G_al[0:10]

[[5321, 2454],
 [1008, 1357],
 [3200, 4320],
 [5578, 2101],
 [1085, 1828, 7151, 1828],
 [7551],
 [4830, 3731],
 [3659, 2939],
 [1685, 7096, 8285],
 [3790, 847]]

# Tests

In [9]:
def commonStreet(ls1, ls2):
    for street in ls1:
        if street in ls2:
            return True
    return False

In [10]:
assert commonStreet(streets[nodeToIntercept[G_al[0][0]]], streets[nodeToIntercept[0]]) and commonStreet(streets[nodeToIntercept[G_al[0][1]]], streets[nodeToIntercept[0]])

In [13]:
assert commonStreet(streets[nodeToIntercept[G_al[0][1]]], streets[nodeToIntercept[0]]) and commonStreet(streets[nodeToIntercept[G_al[0][1]]], streets[nodeToIntercept[0]])

In [14]:
assert commonStreet(streets[nodeToIntercept[G_al[1][0]]], streets[nodeToIntercept[1]]) and commonStreet(streets[nodeToIntercept[G_al[0][1]]], streets[nodeToIntercept[0]])

# Writing the graph adjacency list

In [11]:
f = open("graph.al", "w")
c = 0
for line in G_al:
    f.write(str(c)+' ')
    if len(line) > 0:
        for x in line:
            f.write(str(x)+' ')
    else:
        f.write('-')
    f.write('\n')
    c += 1